In [1]:
# Importaciones necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [2]:
# Echemos un vistazo preliminar al fichero
!cat "Ficheros/Manchas/SN_d_tot_V2.0.csv" | head

1818;01;01;1818.001;  -1; -1.0;   0;1
1818;01;02;1818.004;  -1; -1.0;   0;1
1818;01;03;1818.007;  -1; -1.0;   0;1
1818;01;04;1818.010;  -1; -1.0;   0;1
1818;01;05;1818.012;  -1; -1.0;   0;1
1818;01;06;1818.015;  -1; -1.0;   0;1
1818;01;07;1818.018;  -1; -1.0;   0;1
1818;01;08;1818.021;  65; 10.2;   1;1
1818;01;09;1818.023;  -1; -1.0;   0;1
1818;01;10;1818.026;  -1; -1.0;   0;1
cat: write error: Broken pipe


In [ ]:
# La estructura del fichero es:
# Line format [character position]:
# [1-4] Year
# [6-7] Month
# [9-10] Day
# [12-19] Decimal date
# [22-24] Daily sunspot number
# [26-30] Standard deviation
# [33-35] Number of observations
# [37] Definitive/provisional indicator

In [3]:
# Con la descripción de los datos que tenemos (ver "SN_d_tot_V2.0_description.txt") y 
# sabiendo que los NaN se coresponden con el valor -1 (y que también tenemos 0) podemos ya cargarlo:
columns = ['Year', 'Month', 'Day', 'Fraction', 'Spots', 'Stdev', 'Observ', 'Def/Prov']
df = pd.read_csv('Ficheros/Manchas/SN_d_tot_V2.0.csv', sep = ';', header = None, names = columns, na_values = [0, -1])
df.head(10)

,Year,Month,Day,Fraction,Spots,Stdev,Observ,Def/Prov
0,1818,1,1,1818.001,-1,NaN,0,1.0
1,1818,1,2,1818.004,-1,NaN,0,1.0
2,1818,1,3,1818.007,-1,NaN,0,1.0
3,1818,1,4,1818.010,-1,NaN,0,1.0
4,1818,1,5,1818.012,-1,NaN,0,1.0
5,1818,1,6,1818.015,-1,NaN,0,1.0
6,1818,1,7,1818.018,-1,NaN,0,1.0
7,1818,1,8,1818.021,65,10.2,1,1.0
8,1818,1,9,1818.023,-1,NaN,0,1.0
9,1818,1,10,1818.026,-1,NaN,0,1.0


In [4]:
# No parece que nos haya transformado en NaN los datos de 'Spots', debe haber espacios en esa columna,
# vamos a quitarlos
columns = ['Year', 'Month', 'Day', 'Fraction', 'Spots', 'Stdev', 'Observ', 'Def/Prov']
df = pd.read_csv('Ficheros/Manchas/SN_d_tot_V2.0.csv', sep = r'\s*;\s*', header = None, names = columns, na_values =[0, -1],
                engine = 'python')
df.head(10)

,Year,Month,Day,Fraction,Spots,Stdev,Observ,Def/Prov
0,1818,1,1,1818.001,NaN,NaN,NaN,1.0
1,1818,1,2,1818.004,NaN,NaN,NaN,1.0
2,1818,1,3,1818.007,NaN,NaN,NaN,1.0
3,1818,1,4,1818.010,NaN,NaN,NaN,1.0
4,1818,1,5,1818.012,NaN,NaN,NaN,1.0
5,1818,1,6,1818.015,NaN,NaN,NaN,1.0
6,1818,1,7,1818.018,NaN,NaN,NaN,1.0
7,1818,1,8,1818.021,65.0,10.2,1.0,1.0
8,1818,1,9,1818.023,NaN,NaN,NaN,1.0
9,1818,1,10,1818.026,NaN,NaN,NaN,1.0


In [9]:
# Un poco de información
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73839 entries, 0 to 73838
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Year      73839 non-null  int64  
 1   Month     73839 non-null  int64  
 2   Day       73839 non-null  int64  
 3   Fraction  73839 non-null  float64
 4   Spots     59396 non-null  float64
 5   Stdev     67975 non-null  float64
 6   Observ    70592 non-null  float64
 7   Def/Prov  73687 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 4.5 MB
None
               Year         Month           Day      Fraction         Spots  \
count  73839.000000  73839.000000  73839.000000  73839.000000  59396.000000   
mean    1918.582389      6.518926     15.729195   1919.082049     98.416880   
std       58.360030      3.450319      8.799946     58.360155     74.550474   
min     1818.000000      1.000000      1.000000   1818.001000      1.000000   
25%     1868.000000      4.000000      8.000000   18

In [10]:
# Controlamos los NaN
df.isna().sum()


Year            0
Month           0
Day             0
Fraction        0
Spots       14443
Stdev        5864
Observ       3247
Def/Prov      152
dtype: int64

In [12]:
# Lo que nos interesa está en la columna 'Spots', eliminamos filas con NaN en dicha columna
df = df.dropna(axis=0, subset=['Spots'])

In [13]:
# Crearemos una columna en formato fecha para operaciones posteriores
df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

In [15]:
df.describe()

,Year,Month,Day,Fraction,Spots,Stdev,Observ,Def/Prov
count,59396.000000,59396.000000,59396.000000,59396.000000,59396.000000,59391.000000,59396.000000,59368.0
mean,1924.847262,6.526163,15.721698,1925.347479,98.416880,8.361078,4.557293,1.0
std,55.665937,3.417239,8.812485,55.665414,74.550474,4.162083,7.588013,0.0
min,1818.000000,1.000000,1.000000,1818.021000,1.000000,0.200000,1.000000,1.0
25%,1877.000000,4.000000,8.000000,1877.028250,37.000000,5.300000,1.000000,1.0
50%,1928.000000,7.000000,16.000000,1928.377000,82.000000,7.900000,1.000000,1.0
75%,1973.000000,9.000000,23.000000,1973.163750,142.000000,10.900000,1.000000,1.0
max,2020.000000,12.000000,31.000000,2020.086000,528.000000,77.700000,58.000000,1.0


In [16]:
# Tomaremos como outliers la media + 3Stdev (~325)
len(df[df['Spots'] > 325])

571

In [17]:
# Los borramos
df.drop(df[df.Spots > 325].index, inplace =True)

In [18]:
# Solo nos interesan la nueva columna de fecha y el número de manchas (Spots). Eliminamos el resto
df.drop(['Year', 'Month', 'Day', 'Fraction', 'Stdev', 'Observ', 'Def/Prov'], axis = 'columns', inplace = True)

In [20]:
# Reordenamos
df = df[['Date', 'Spots']]

In [22]:
# Listo! Lo guardamos para más tarde
df.to_csv('Spots_new.csv', index = False)